In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
if not (os.path.exists('./data/userid-profile.tsv') and os.path.exists('./data/userid-timestamp-artid-artname-traid-traname.tsv')):
  raise ValueError('Los archivos userid-profile y userid-timestamp-artid-artname-traid-traname no fueron encontrados en el path')
else:
  print("Los archivos han sido cargados")

In [ ]:
users=pd.read_csv('./data/userid-profile.tsv', sep = '\t', header=0 )

In [ ]:
activity=pd.read_csv('./data/userid-timestamp-artid-artname-traid-traname.tsv', sep = '\t', names = [ 'userid', 'timestamp', 'musicbrainz-artist-id', 'artist-name', 'musicbrainz-track-id', 'track-name' ])

### Primero removemos los 50 usuarios con menos interacciones para reducir el tamaño del dataframe

In [ ]:
activity_lastFifty = activity.groupby('userid')['artist-name'].agg({'count'}).sort_values(by='count', ascending=False).tail(50)

In [ ]:
activity_lastFifty_union = activity.merge(activity_lastFifty, how='inner', left_on='userid', right_index=True)
activity_lastFifty_union = activity_lastFifty_union[['userid', 'timestamp', 'musicbrainz-artist-id', 'artist-name', 'musicbrainz-track-id', 'track-name']]

In [ ]:
dataframes = [activity,activity_lastFifty_union]
reduced_activity = pd.concat(dataframes).drop_duplicates(keep=False)

### Luego contamos las veces en las que un usuario escuchó un artista

In [ ]:
user_artists_count = reduced_activity.groupby(['userid', 'artist-name']).count()

In [ ]:
sorted_user_artists_count = user_artists_count['track-name'].reset_index(name='count').sort_values(['userid', 'count'], ascending=False)
sorted_user_artists_count['rating'] = 0

### Por último sacamos un rating basado en la cantidad de veces que un artista fué escuchado de manera proporcional

In [ ]:
pd.options.mode.chained_assignment = None

In [222]:
test = sorted_user_artists_count.head(100)

In [223]:
def calculateRating (row):
    userid = row['userid']
    userdf = sorted_user_artists_count[sorted_user_artists_count['userid'] == userid]
    countmax = userdf['count'].max()
    rating = 10 * row['count']/countmax
    return rating

### Reemplazar "test" con sorted_user_artists_count

In [224]:
test['rating'] = test.apply(lambda x: calculateRating(x), axis=1)
test

,userid,artist-name,count,rating
894480,user_001000,Wilco,455,10.000000
894278,user_001000,Radiohead,444,9.758242
893852,user_001000,Animal Collective,432,9.494505
894021,user_001000,Girl Talk,417,9.164835
893834,user_001000,Aesop Rock,383,8.417582
...,...,...,...,...
893901,user_001000,Broken Social Scene,59,1.296703
894369,user_001000,Tapes 'N Tapes,58,1.274725
894386,user_001000,The Blow,58,1.274725
894453,user_001000,Tv On The Radio,58,1.274725
